In [ ]:
#Using Google Drive for loading the data(uploaded all the datafiles to drive  under deep-learning-for-msc-coursework-2022-2 folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing all the required packages
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pathlib
import glob


import random

random.seed(100)

torch.manual_seed(100)

from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

In [ ]:
trainroot = "/content/drive/My Drive/deep-learning-for-msc-coursework-2022-2/train/train"
testroot = "/content/drive/My Drive/deep-learning-for-msc-coursework-2022-2/test"
RANDOM_SEED = 123
# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 64

# Spatial size of training images which is 32*32 

image_size = 32

# Learning rate for optimizers
lr = 0.0002


# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:

dataset = dset.ImageFolder(root=trainroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
print(len(dataset))
train_len = len(dataset) - 230
train_set, val_set = torch.utils.data.random_split(dataset, [train_len, 230])
train_data = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
 
                                         shuffle=True, )
print(len(train_data))
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

1730
24


In [ ]:
### Examining class names
root = pathlib.Path(trainroot)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes

['Cancer', 'Connective', 'Immune', 'Normal']

In [ ]:

## CNN Network
class ConvNet(nn.Module):
  def __init__(self,num_classes = 4):
    super(ConvNet,self).__init__()

    #Input shape = (32,3,32,32)
    #1
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3,stride = 1, padding =1)
    #shape = (32,12,32,32)
    self.bn1= nn.BatchNorm2d(num_features = 12)
    self.relu1 = nn.ReLU()
    #shape = (32,12,32,32)
    #2
    self.pool = nn.MaxPool2d(kernel_size = 2)
    #shape = (32,12,16,16)
    #3
    self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 20, kernel_size = 3,stride = 1, padding =1)
    self.relu2 = nn.ReLU()
    #4
    
    self.conv3 = nn.Conv2d(in_channels = 20, out_channels = 20, kernel_size = 3,stride = 1, padding =1)
    self.relu3 = nn.ReLU()
   
    #5
    self.conv4 = nn.Conv2d(in_channels = 20, out_channels = 20, kernel_size = 3,stride = 1, padding =1)
    self.relu4 = nn.ReLU()
   
   #6
    self.conv5 = nn.Conv2d(in_channels = 20, out_channels = 32, kernel_size = 3,stride = 1, padding =1)
    #shape = (32,12,32,32)
    self.bn5= nn.BatchNorm2d(num_features = 32)
    self.relu5 = nn.ReLU()
    #shape = (32,12,16,16)
    # Fully connected #7
    self.fc = nn.Linear(in_features = 32*16*16, out_features = num_classes)
    


  def forward(self,input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.relu1(output)

    output = self.pool(output)

    output = self.conv2(output)
    output = self.relu2(output)
   
  
    output = self.conv3(output)
    output = self.relu3(output)
  

    output = self.conv4(output)
    output = self.relu4(output)
 

    output = self.conv5(output)
    output = self.bn5(output)
    output = self.relu5(output)   
    


    output= output.view(-1,32*16*16)
    output = self.fc(output)
    return output


In [ ]:
#Creating the model
model = ConvNet(num_classes = 4).to(device)

In [ ]:

#Using Stochastic Gradient Descent for the optimizer
optimizer = optim.SGD(model.parameters(), lr=lr)
#using CrossEntropy Loss for the cross function and setting up the weights for the 4 classes
loss_function = nn.CrossEntropyLoss(weight = torch.tensor([1,1,1,1.5]))

In [ ]:
import datetime
epoch_print_gap = 1
#Defining the training function(referred lab material)
def training_loop(n_epochs, optimizer, model, device, loss_fn, train_loader):
    model = model.to(device)
    for epoch in range(1, n_epochs + 1):
      
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs.to(device))
            loss = loss_fn(outputs, labels.to(device))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
            
        if epoch == 1 or epoch % epoch_print_gap == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch, float(loss_train)))

In [ ]:
#Training the model
training_loop(
    n_epochs = 11, 
    optimizer = optimizer,
    model = model, 
    device = device,
    loss_fn = loss_function,
    train_loader = train_data,
)


2022-03-25 14:33:54.515780 Epoch 1, Training loss 31.9187833070755
2022-03-25 14:34:01.003705 Epoch 2, Training loss 26.37728577852249
2022-03-25 14:34:06.980565 Epoch 3, Training loss 22.567627608776093
2022-03-25 14:34:12.350607 Epoch 4, Training loss 19.64997363090515
2022-03-25 14:34:17.939874 Epoch 5, Training loss 17.633263170719147
2022-03-25 14:34:23.052881 Epoch 6, Training loss 15.855398058891296
2022-03-25 14:34:27.915600 Epoch 7, Training loss 14.834021508693695
2022-03-25 14:34:32.867570 Epoch 8, Training loss 13.644444465637207
2022-03-25 14:34:37.670852 Epoch 9, Training loss 12.788468182086945
2022-03-25 14:34:42.682320 Epoch 10, Training loss 12.351852983236313
2022-03-25 14:34:47.615044 Epoch 11, Training loss 11.481696248054504


In [ ]:
#Defining the validation loop (referred lab material)
def validation_loop(model, device, test_loader):
    model.eval()
    model = model.to(device)
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
       
            test_loss += torch.nn.functional.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
#Loading the validation set
val_data = torch.utils.data.DataLoader(val_set)
 
# Not using RandomHorizontalFlip as we don't require any kind of variation while we are testing                                    )
transformer=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),  
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] 
                        [0.5,0.5,0.5])
])
validation_loop(model = model, device = device, test_loader = val_data)



Test set: Average loss: 0.4909, Accuracy: 201/230 (87%)



In [ ]:
#Defining the prediction function
def predict_loop(img_path,transformer):
    
    images=Image.open(img_path)
    
    image_tranformed=transformer(images).float()
    
    
    image_tranformed=image_tranformed.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tranformed.cuda()
        
    input_variable=Variable(image_tranformed)
    
    output_val=model(input_variable)
    
    index_val=output_val.data.numpy().argmax()
    
    pred=classes[index_val]
    
    return pred

In [ ]:
#Using the images path
images_path=glob.glob(testroot+'/test'+ '/*.png')


In [ ]:
#Using the Transforms similar to the validation set
transformer=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),  
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])


prediction_dict={}
#Storing all the predicted values to the dictionary
for i in images_path:
    prediction_dict[i[i.rfind('/')+1:]]=predict_loop(i,transformer)

In [ ]:
#printing out the keys
prediction_dict.keys()

dict_keys(['tile7059.png', 'tile7134.png', 'tile7212.png', 'tile7014.png', 'tile7047.png', 'tile7151.png', 'tile7039.png', 'tile7095.png', 'tile7044.png', 'tile7223.png', 'tile7116.png', 'tile7068.png', 'tile7132.png', 'tile7110.png', 'tile7181.png', 'tile7137.png', 'tile7027.png', 'tile7067.png', 'tile7043.png', 'tile7013.png', 'tile7228.png', 'tile7147.png', 'tile7155.png', 'tile7078.png', 'tile7054.png', 'tile7020.png', 'tile7009.png', 'tile7090.png', 'tile7136.png', 'tile7037.png', 'tile7081.png', 'tile7237.png', 'tile7225.png', 'tile7156.png', 'tile7204.png', 'tile7108.png', 'tile7213.png', 'tile7154.png', 'tile7006.png', 'tile7102.png', 'tile7606.png', 'tile7473.png', 'tile7457.png', 'tile7638.png', 'tile7454.png', 'tile7541.png', 'tile7564.png', 'tile7559.png', 'tile7615.png', 'tile7480.png', 'tile7589.png', 'tile7374.png', 'tile7540.png', 'tile7437.png', 'tile7527.png', 'tile7324.png', 'tile7420.png', 'tile7600.png', 'tile7568.png', 'tile7572.png', 'tile7471.png', 'tile7630.png

In [ ]:
#Storing the predicted values in the CSV 
from google.colab import files
import pandas as pd
result = pd.DataFrame(prediction_dict.items(), columns=['Id', 'Type'])

result.to_csv('prediction.csv',index=False)
!cp prediction.csv "/content/drive/My Drive/"

In [ ]:
#Distribution of Each type 
result['Type'].value_counts() 

Cancer        1061
Connective     942
Normal         601
Immune         596
Name: Type, dtype: int64